In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
import numpy as np
from numba import njit, jit
import pandas as pd
from collections import deque
from numba import njit
import matplotlib.pyplot as plt

In [2]:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
ALL_PAIRS = ['EURCHF','EURGBP','EURJPY','EURNZD','EURAUD','EURCAD',
              'GBPAUD','GBPCHF','GBPJPY','GBPCAD','GBPUSD','GBPNZD','USDCHF',
              'USDJPY','AUDUSD','NZDUSD','USDCAD','AUDJPY','CADJPY','CHFJPY',
              'NZDJPY','AUDCHF','CADCHF','NZDCHF','AUDNZD','NZDCAD','AUDCAD']
datam1 = pd.read_csv('/content/drive/MyDrive/My_Project/Colab Teste/Dados/EURUSD.csv', sep=',').drop(columns=['Unnamed: 0'])
for i in ALL_PAIRS:
  x = pd.read_csv(f'/content/drive/MyDrive/My_Project/Colab Teste/Dados/{i}.csv', sep=',').drop(columns=['Unnamed: 0'])
  datam1 = pd.concat([datam1, x], axis=1)

In [4]:
datam1.isna().sum().sum()

0

In [5]:
datam1

,EURUSD_Open,EURUSD_Close,EURUSD_High,EURUSD_Low,EURCHF_Open,EURCHF_Close,EURCHF_High,EURCHF_Low,EURGBP_Open,EURGBP_Close,EURGBP_High,EURGBP_Low,EURJPY_Open,EURJPY_Close,EURJPY_High,EURJPY_Low,EURNZD_Open,EURNZD_Close,EURNZD_High,EURNZD_Low,EURAUD_Open,EURAUD_Close,EURAUD_High,EURAUD_Low,EURCAD_Open,EURCAD_Close,EURCAD_High,EURCAD_Low,GBPAUD_Open,GBPAUD_Close,GBPAUD_High,GBPAUD_Low,GBPCHF_Open,GBPCHF_Close,GBPCHF_High,GBPCHF_Low,GBPJPY_Open,GBPJPY_Close,GBPJPY_High,GBPJPY_Low,...,AUDJPY_Open,AUDJPY_Close,AUDJPY_High,AUDJPY_Low,CADJPY_Open,CADJPY_Close,CADJPY_High,CADJPY_Low,CHFJPY_Open,CHFJPY_Close,CHFJPY_High,CHFJPY_Low,NZDJPY_Open,NZDJPY_Close,NZDJPY_High,NZDJPY_Low,AUDCHF_Open,AUDCHF_Close,AUDCHF_High,AUDCHF_Low,CADCHF_Open,CADCHF_Close,CADCHF_High,CADCHF_Low,NZDCHF_Open,NZDCHF_Close,NZDCHF_High,NZDCHF_Low,AUDNZD_Open,AUDNZD_Close,AUDNZD_High,AUDNZD_Low,NZDCAD_Open,NZDCAD_Close,NZDCAD_High,NZDCAD_Low,AUDCAD_Open,AUDCAD_Close,AUDCAD_High,AUDCAD_Low
0,1.34551,1.34560,1.34579,1.34536,1.30206,1.30207,1.30213,1.30193,0.85107,0.85095,0.85107,0.85094,114.996,114.978,114.998,114.978,1.85060,1.85076,1.85095,1.85049,1.42439,1.42434,1.42443,1.42433,1.40441,1.40455,1.40459,1.40439,1.61753,1.61778,1.61786,1.61732,1.54155,1.54231,1.54239,1.54137,132.215,132.256,132.263,132.215,...,80.999,80.998,81.008,80.997,81.037,81.022,81.038,81.010,85.619,85.614,85.635,85.609,62.044,62.045,62.047,62.043,0.94377,0.94387,0.94387,0.94377,0.97279,0.97266,0.97289,0.97264,0.73507,0.73515,0.73518,0.73507,1.30624,1.30628,1.30631,1.30613,0.74928,0.74941,0.74947,0.74926,0.99195,0.99165,0.99195,0.99165
1,1.34559,1.34562,1.34566,1.34533,1.30203,1.30203,1.30204,1.30188,0.85096,0.85104,0.85104,0.85093,114.977,114.993,114.993,114.952,1.85079,1.85142,1.85148,1.85073,1.42437,1.42441,1.42453,1.42435,1.40456,1.40445,1.40457,1.40441,1.61779,1.61818,1.61819,1.61777,1.54226,1.54194,1.54239,1.54193,132.255,132.245,132.256,132.239,...,80.999,80.989,81.005,80.989,81.027,81.043,81.047,81.019,85.613,85.638,85.639,85.604,62.046,62.053,62.053,62.044,0.94385,0.94385,0.94390,0.94385,0.97267,0.97291,0.97299,0.97264,0.73512,0.73520,0.73523,0.73512,1.30631,1.30624,1.30643,1.30619,0.74942,0.74910,0.74942,0.74910,0.99163,0.99162,0.99175,0.99158
2,1.34564,1.34583,1.34583,1.34564,1.30204,1.30247,1.30254,1.30204,0.85103,0.85107,0.85109,0.85101,114.990,114.986,114.995,114.976,1.85141,1.85112,1.85144,1.85112,1.42443,1.42442,1.42453,1.42437,1.40444,1.40446,1.40463,1.40439,1.61815,1.61754,1.61828,1.61739,1.54193,1.54199,1.54209,1.54193,132.248,132.278,132.279,132.234,...,80.991,80.993,80.996,80.977,81.040,81.036,81.045,81.032,85.637,85.685,85.685,85.633,62.054,62.067,62.067,62.053,0.94386,0.94387,0.94390,0.94386,0.97287,0.97277,0.97288,0.97277,0.73516,0.73518,0.73523,0.73509,1.30622,1.30630,1.30648,1.30622,0.74907,0.74893,0.74907,0.74892,0.99163,0.99167,0.99178,0.99160
3,1.34582,1.34589,1.34600,1.34580,1.30245,1.30274,1.30277,1.30244,0.85106,0.85102,0.85107,0.85101,114.988,115.024,115.025,114.985,1.85109,1.85151,1.85195,1.85088,1.42439,1.42449,1.42449,1.42422,1.40448,1.40437,1.40452,1.40437,1.61749,1.61721,1.61749,1.61707,1.54196,1.54194,1.54206,1.54167,132.276,132.290,132.308,132.272,...,80.994,80.990,80.996,80.988,81.037,81.035,81.039,81.032,85.682,85.670,85.692,85.668,62.069,62.102,62.102,62.069,0.94388,0.94386,0.94388,0.94386,0.97281,0.97307,0.97307,0.97281,0.73521,0.73508,0.73522,0.73505,1.30628,1.30624,1.30648,1.30614,0.74892,0.74887,0.74899,0.74887,0.99171,0.99179,0.99184,0.99171
4,1.34584,1.34585,1.34599,1.34574,1.30273,1.30262,1.30273,1.30259,0.85103,0.85100,0.85106,0.85096,115.022,115.042,115.042,115.020,1.85157,1.85176,1.85207,1.85125,1.42450,1.42452,1.42454,1.42437,1.40436,1.40438,1.40444,1.40428,1.61723,1.61749,1.61751,1.61698,1.54197,1.54203,1.54211,1.54186,132.288,132.250,132.288,132.221,...,80.988,80.984,80.993,80.983,81.033,81.032,81.033,81.026,85.672,85.669,85.673,85.629,62.105,62.113,62.123,62.104,0.94389,0.94389,0.94389,0.94389,0.97303,0.97335,0.97352,0.97303,0.73506,0.735

In [6]:
datam1.to_csv('Data_M1_3_9kk_icmarkets.csv',sep=',')

In [ ]:
datam1 = pd.read_csv('/content/drive/MyDrive/My_Project/Colab Teste/Dados/DataM13kk.csv', sep=',').drop(columns=['Unnamed: 0'])
datam1

,EURUSD_Open,EURUSD_Close,EURUSD_High,EURUSD_Low,EURCHF_Open,EURCHF_Close,EURCHF_High,EURCHF_Low,EURGBP_Open,EURGBP_Close,EURGBP_High,EURGBP_Low,EURJPY_Open,EURJPY_Close,EURJPY_High,EURJPY_Low,EURNZD_Open,EURNZD_Close,EURNZD_High,EURNZD_Low,EURAUD_Open,EURAUD_Close,EURAUD_High,EURAUD_Low,EURCAD_Open,EURCAD_Close,EURCAD_High,EURCAD_Low,GBPAUD_Open,GBPAUD_Close,GBPAUD_High,GBPAUD_Low,GBPCHF_Open,GBPCHF_Close,GBPCHF_High,GBPCHF_Low,GBPJPY_Open,GBPJPY_Close,GBPJPY_High,GBPJPY_Low,...,AUDJPY_Open,AUDJPY_Close,AUDJPY_High,AUDJPY_Low,CADJPY_Open,CADJPY_Close,CADJPY_High,CADJPY_Low,CHFJPY_Open,CHFJPY_Close,CHFJPY_High,CHFJPY_Low,NZDJPY_Open,NZDJPY_Close,NZDJPY_High,NZDJPY_Low,AUDCHF_Open,AUDCHF_Close,AUDCHF_High,AUDCHF_Low,CADCHF_Open,CADCHF_Close,CADCHF_High,CADCHF_Low,NZDCHF_Open,NZDCHF_Close,NZDCHF_High,NZDCHF_Low,AUDNZD_Open,AUDNZD_Close,AUDNZD_High,AUDNZD_Low,NZDCAD_Open,NZDCAD_Close,NZDCAD_High,NZDCAD_Low,AUDCAD_Open,AUDCAD_Close,AUDCAD_High,AUDCAD_Low
0,1.31203,1.31177,1.31216,1.31152,1.25149,1.25158,1.25160,1.25147,0.84312,0.84306,0.84312,0.84306,112.503,112.516,112.517,112.502,1.78295,1.78318,1.78318,1.78295,1.37555,1.37561,1.37571,1.37555,1.34518,1.34556,1.34571,1.34516,1.60505,1.60405,1.60505,1.60363,1.52619,1.52505,1.52623,1.52501,132.147,132.151,132.151,132.145,...,81.863,81.839,81.868,81.832,83.167,83.229,83.241,83.167,85.739,85.742,85.742,85.738,63.717,63.709,63.717,63.709,0.95368,0.95364,0.95368,0.95342,0.97661,0.97664,0.97687,0.97660,0.72785,0.72798,0.72798,0.72781,1.29554,1.29539,1.29567,1.29538,0.76177,0.76170,0.76180,0.76170,0.99190,0.99201,0.99201,0.99161
1,1.31174,1.31168,1.31175,1.31143,1.25160,1.25162,1.25163,1.25157,0.84305,0.84334,0.84335,0.84305,112.516,112.513,112.531,112.512,1.78315,1.78341,1.78341,1.78309,1.37563,1.37550,1.37567,1.37550,1.34555,1.34549,1.34563,1.34527,1.60401,1.60439,1.60462,1.60378,1.52506,1.52512,1.52525,1.52499,132.152,132.133,132.153,132.133,...,81.838,81.841,81.857,81.838,83.223,83.225,83.225,83.159,85.741,85.749,85.761,85.741,63.709,63.705,63.713,63.705,0.95365,0.95357,0.95367,0.95353,0.97663,0.97648,0.97674,0.97647,0.72798,0.72809,0.72812,0.72794,1.29540,1.29545,1.29556,1.29532,0.76176,0.76175,0.76176,0.76171,0.99198,0.99199,0.99214,0.99197
2,1.31167,1.31198,1.31205,1.31167,1.25161,1.25162,1.25164,1.25158,0.84335,0.84334,0.84336,0.84332,112.513,112.502,112.513,112.497,1.78343,1.78345,1.78354,1.78339,1.37551,1.37550,1.37561,1.37545,1.34545,1.34600,1.34607,1.34530,1.60438,1.60462,1.60489,1.60431,1.52514,1.52569,1.52585,1.52512,132.134,132.117,132.134,132.117,...,81.842,81.847,81.854,81.834,83.225,83.234,83.272,83.221,85.748,85.717,85.749,85.717,63.707,63.695,63.707,63.695,0.95358,0.95384,0.95397,0.95358,0.97648,0.97689,0.97689,0.97647,0.72809,0.72825,0.72825,0.72809,1.29542,1.29549,1.29553,1.29537,0.76174,0.76184,0.76184,0.76173,0.99200,0.99205,0.99208,0.99197
3,1.31193,1.31215,1.31218,1.31183,1.25158,1.25148,1.25158,1.25148,0.84333,0.84337,0.84342,0.84329,112.500,112.496,112.505,112.490,1.78347,1.78320,1.78347,1.78295,1.37551,1.37548,1.37557,1.37537,1.34606,1.34623,1.34625,1.34592,1.60465,1.60462,1.60507,1.60457,1.52566,1.52600,1.52604,1.52566,132.116,132.124,132.128,132.114,...,81.854,81.834,81.866,81.826,83.237,83.281,83.306,83.235,85.718,85.735,85.746,85.716,63.699,63.695,63.699,63.695,0.95386,0.95386,0.95396,0.95382,0.97685,0.97664,0.97685,0.97657,0.72822,0.72828,0.72844,0.72817,1.29551,1.29533,1.29555,1.29533,0.76184,0.76195,0.76217,0.76178,0.99178,0.99199,0.99199,0.99178
4,1.31212,1.31219,1.31222,1.31201,1.25147,1.25146,1.25154,1.25145,0.84336,0.84347,0.84347,0.84333,112.497,112.499,112.504,112.496,1.78317,1.78321,1.78324,1.78315,1.37546,1.37511,1.37546,1.37482,1.34620,1.34580,1.34624,1.34558,1.60473,1.60535,1.60571,1.60473,1.52603,1.52605,1.52611,1.52570,132.125,132.121,132.127,132.116,...,81.833,81.850,81.855,81.833,83.280,83.270,83.302,83.270,85.734,85.748,85.752,85.726,63.695,63.695,63.695,63.695,0.95387,0.95394,0.95403,0.95387,0.97663,0.97621,0.97663,0.97616,0.72828,0.728

## Funções

In [ ]:
ALL_PAIRS_BUY = np.array([
    'EURCHF_buy','EURGBP_buy','EURJPY_buy','EURNZD_buy','EURUSD_buy','EURAUD_buy','EURCAD_buy',
    'GBPAUD_buy','GBPCHF_buy','GBPJPY_buy','GBPCAD_buy','GBPUSD_buy','GBPNZD_buy','USDCHF_buy',
    'USDJPY_buy','AUDUSD_buy','NZDUSD_buy','USDCAD_buy','AUDJPY_buy','CADJPY_buy','CHFJPY_buy',
    'NZDJPY_buy','AUDCHF_buy','CADCHF_buy','NZDCHF_buy','AUDNZD_buy','NZDCAD_buy','AUDCAD_buy'
])

ALL_PAIRS_SELL = np.array([
    'EURCHF_sell','EURGBP_sell','EURJPY_sell','EURNZD_sell','EURUSD_sell','EURAUD_sell','EURCAD_sell',
    'GBPAUD_sell','GBPCHF_sell','GBPJPY_sell','GBPCAD_sell','GBPUSD_sell','GBPNZD_sell','USDCHF_sell',
    'USDJPY_sell','AUDUSD_sell','NZDUSD_sell','USDCAD_sell','AUDJPY_sell','CADJPY_sell','CHFJPY_sell',
    'NZDJPY_sell','AUDCHF_sell','CADCHF_sell','NZDCHF_sell','AUDNZD_sell','NZDCAD_sell','AUDCAD_sell'
])

SPLIT_PAIRS = np.array([
    ['eur','chf'],['eur','gbp'],['eur','jpy'],['eur','nzd'],['eur','usd'],
    ['eur','aud'],['eur','cad'],['gbp','aud'],['gbp','chf'],['gbp','jpy'],
    ['gbp','cad'],['gbp','usd'],['gbp','nzd'],['usd','chf'],['usd','jpy'],
    ['aud','usd'],['nzd','usd'],['usd','cad'],['aud','jpy'],['cad','jpy'],
    ['chf','jpy'],['nzd','jpy'],['aud','chf'],['cad','chf'],['nzd','chf'],
    ['aud','nzd'],['nzd','cad'],['aud','cad']
])

def pct_data(data, period=1, complement='_Open'):
    """
    No futuro colcoar a soma, subtração de todos os 4
    """

    diff_symbols = {
        'eur':['EURCHF','EURGBP','EURJPY','EURNZD','EURUSD','EURAUD','EURCAD'],
        'gbp':['EURGBP','GBPAUD','GBPCHF','GBPJPY','GBPCAD','GBPUSD','GBPNZD'],
        'usd':['GBPUSD','USDCHF','USDJPY','AUDUSD','NZDUSD','USDCAD','EURUSD'],
        'jpy':['AUDJPY','CADJPY','CHFJPY','EURJPY','USDJPY','GBPJPY','NZDJPY'],
        'chf':['AUDCHF','CADCHF','CHFJPY','USDCHF','EURCHF','GBPCHF','NZDCHF'],
        'nzd':['AUDNZD','EURNZD','GBPNZD','NZDUSD','NZDCAD','NZDCHF','NZDJPY'],
        'aud':['AUDCAD','AUDCHF','AUDJPY','AUDUSD','AUDNZD','EURAUD','GBPAUD'],
        'cad':['AUDCAD','CADCHF','CADJPY','USDCAD','EURCAD','GBPCAD','NZDCAD']
    }

    for i in diff_symbols.items():
        df = pd.DataFrame()
        for j in i[1]:
            df[j]=data[j+complement]
            if i[0] == 'gbp':
                if j == 'EURGBP':
                    df[j] = 1 / df[j]
            if i[0] == 'usd':
                if j in ['GBPUSD','AUDUSD','NZDUSD','EURUSD']:
                    df[j] = 1 / df[j]
            if i[0] == 'jpy':
                df[j] = 1 / df[j]
            if i[0] == 'chf':
                if j in ['AUDCHF','CADCHF','USDCHF','EURCHF','GBPCHF','NZDCHF']:
                    df[j] = 1 / df[j]
            if i[0] == 'nzd':
                if j in ['AUDNZD','EURNZD','GBPNZD']:
                    df[j] = 1 / df[j]
            if i[0] == 'aud':
                if j in ['EURAUD','GBPAUD']:
                    df[j] = 1 / df[j]
            if i[0] == 'cad':
                if j in ['AUDCAD','USDCAD','EURCAD','GBPCAD','NZDCAD']:
                    df[j] = 1 / df[j]
        data[f'{i[0]}'] = df.pct_change(periods=period).sum(axis=1)*100

    return data


def pandas_to_array_massive(data):

    array = np.array(
            [
                np.transpose(np.array(data[['EURUSD_Open','EURUSD_High','EURUSD_Low','EURUSD_Close']])),
                np.transpose(np.array(data[['EURCHF_Open','EURCHF_High','EURCHF_Low','EURCHF_Close']])),
                np.transpose(np.array(data[['EURGBP_Open','EURGBP_High','EURGBP_Low','EURGBP_Close']])),
                np.transpose(np.array(data[['EURJPY_Open','EURJPY_High','EURJPY_Low','EURJPY_Close']])),
                np.transpose(np.array(data[['EURNZD_Open','EURNZD_High','EURNZD_Low','EURNZD_Close']])),
                np.transpose(np.array(data[['EURAUD_Open','EURAUD_High','EURAUD_Low','EURAUD_Close']])),
                np.transpose(np.array(data[['EURCAD_Open','EURCAD_High','EURCAD_Low','EURCAD_Close']])),
                np.transpose(np.array(data[['GBPAUD_Open','GBPAUD_High','GBPAUD_Low','EURGBP_Close']])),
                np.transpose(np.array(data[['GBPCHF_Open','GBPCHF_High','GBPCHF_Low','EURGBP_Close']])),
                np.transpose(np.array(data[['GBPJPY_Open','GBPJPY_High','GBPJPY_Low','EURGBP_Close']])),
                np.transpose(np.array(data[['GBPCAD_Open','GBPCAD_High','GBPCAD_Low','EURGBP_Close']])),
                np.transpose(np.array(data[['GBPUSD_Open','GBPUSD_High','GBPUSD_Low','EURGBP_Close']])),
                np.transpose(np.array(data[['GBPNZD_Open','GBPNZD_High','GBPNZD_Low','EURGBP_Close']])),
                np.transpose(np.array(data[['USDCHF_Open','USDCHF_High','USDCHF_Low','EURUSD_Close']])),
                np.transpose(np.array(data[['USDJPY_Open','USDJPY_High','USDJPY_Low','EURUSD_Close']])),
                np.transpose(np.array(data[['AUDUSD_Open','AUDUSD_High','AUDUSD_Low','EURAUD_Close']])),
                np.transpose(np.array(data[['NZDUSD_Open','NZDUSD_High','NZDUSD_Low','EURNZD_Close']])),
                np.transpose(np.array(data[['USDCAD_Open','USDCAD_High','USDCAD_Low','EURUSD_Close']])),
                np.transpose(np.array(data[['AUDJPY_Open','AUDJPY_High','AUDJPY_Low','EURAUD_Close']])),
                np.transpose(np.array(data[['CADJPY_Open','CADJPY_High','CADJPY_Low','EURCAD_Close']])),
                np.transpose(np.array(data[['CHFJPY_Open','CHFJPY_High','CHFJPY_Low','EURCHF_Close']])),
                np.transpose(np.array(data[['NZDJPY_Open','NZDJPY_High','NZDJPY_Low','EURNZD_Close']])),
                np.transpose(np.array(data[['AUDCHF_Open','AUDCHF_High','AUDCHF_Low','EURAUD_Close']])),
                np.transpose(np.array(data[['CADCHF_Open','CADCHF_High','CADCHF_Low','EURCAD_Close']])),
                np.transpose(np.array(data[['NZDCHF_Open','NZDCHF_High','NZDCHF_Low','EURNZD_Close']])),
                np.transpose(np.array(data[['AUDNZD_Open','AUDNZD_High','AUDNZD_Low','EURAUD_Close']])),
                np.transpose(np.array(data[['NZDCAD_Open','NZDCAD_High','NZDCAD_Low','EURNZD_Close']])),
                np.transpose(np.array(data[['AUDCAD_Open','AUDCAD_High','AUDCAD_Low','EURAUD_Close']])),
            ], dtype=np.float64)
    return array


def pandas_to_array(data):

    array = np.array(
            [
                np.transpose(np.array(data[['EURUSD_Open','EURUSD_buy','EURUSD_sell','EURUSD_Open','EURUSD_High','EURUSD_Low']])),
                np.transpose(np.array(data[['EURCHF_Open','EURCHF_buy','EURCHF_sell','EURCHF_Open','EURCHF_High','EURCHF_Low']])),
                np.transpose(np.array(data[['EURGBP_Open','EURGBP_buy','EURGBP_sell','EURGBP_Open','EURGBP_High','EURGBP_Low']])),
                np.transpose(np.array(data[['EURJPY_Open','EURJPY_buy','EURJPY_sell','EURJPY_Open','EURJPY_High','EURJPY_Low']])),
                np.transpose(np.array(data[['EURNZD_Open','EURNZD_buy','EURNZD_sell','EURNZD_Open','EURNZD_High','EURNZD_Low']])),
                np.transpose(np.array(data[['EURAUD_Open','EURAUD_buy','EURAUD_sell','EURAUD_Open','EURAUD_High','EURAUD_Low']])),
                np.transpose(np.array(data[['EURCAD_Open','EURCAD_buy','EURCAD_sell','EURCAD_Open','EURCAD_High','EURCAD_Low']])),
                np.transpose(np.array(data[['GBPAUD_Open','GBPAUD_buy','GBPAUD_sell','EURGBP_Open','GBPAUD_High','GBPAUD_Low']])),
                np.transpose(np.array(data[['GBPCHF_Open','GBPCHF_buy','GBPCHF_sell','EURGBP_Open','GBPCHF_High','GBPCHF_Low']])),
                np.transpose(np.array(data[['GBPJPY_Open','GBPJPY_buy','GBPJPY_sell','EURGBP_Open','GBPJPY_High','GBPJPY_Low']])),
                np.transpose(np.array(data[['GBPCAD_Open','GBPCAD_buy','GBPCAD_sell','EURGBP_Open','GBPCAD_High','GBPCAD_Low']])),
                np.transpose(np.array(data[['GBPUSD_Open','GBPUSD_buy','GBPUSD_sell','EURGBP_Open','GBPUSD_High','GBPUSD_Low']])),
                np.transpose(np.array(data[['GBPNZD_Open','GBPNZD_buy','GBPNZD_sell','EURGBP_Open','GBPNZD_High','GBPNZD_Low']])),
                np.transpose(np.array(data[['USDCHF_Open','USDCHF_buy','USDCHF_sell','EURUSD_Open','USDCHF_High','USDCHF_Low']])),
                np.transpose(np.array(data[['USDJPY_Open','USDJPY_buy','USDJPY_sell','EURUSD_Open','USDJPY_High','USDJPY_Low']])),
                np.transpose(np.array(data[['AUDUSD_Open','AUDUSD_buy','AUDUSD_sell','EURAUD_Open','AUDUSD_High','AUDUSD_Low']])),
                np.transpose(np.array(data[['NZDUSD_Open','NZDUSD_buy','NZDUSD_sell','EURNZD_Open','NZDUSD_High','NZDUSD_Low']])),
                np.transpose(np.array(data[['USDCAD_Open','USDCAD_buy','USDCAD_sell','EURUSD_Open','USDCAD_High','USDCAD_Low']])),
                np.transpose(np.array(data[['AUDJPY_Open','AUDJPY_buy','AUDJPY_sell','EURAUD_Open','AUDJPY_High','AUDJPY_Low']])),
                np.transpose(np.array(data[['CADJPY_Open','CADJPY_buy','CADJPY_sell','EURCAD_Open','CADJPY_High','CADJPY_Low']])),
                np.transpose(np.array(data[['CHFJPY_Open','CHFJPY_buy','CHFJPY_sell','EURCHF_Open','CHFJPY_High','CHFJPY_Low']])),
                np.transpose(np.array(data[['NZDJPY_Open','NZDJPY_buy','NZDJPY_sell','EURNZD_Open','NZDJPY_High','NZDJPY_Low']])),
                np.transpose(np.array(data[['AUDCHF_Open','AUDCHF_buy','AUDCHF_sell','EURAUD_Open','AUDCHF_High','AUDCHF_Low']])),
                np.transpose(np.array(data[['CADCHF_Open','CADCHF_buy','CADCHF_sell','EURCAD_Open','CADCHF_High','CADCHF_Low']])),
                np.transpose(np.array(data[['NZDCHF_Open','NZDCHF_buy','NZDCHF_sell','EURNZD_Open','NZDCHF_High','NZDCHF_Low']])),
                np.transpose(np.array(data[['AUDNZD_Open','AUDNZD_buy','AUDNZD_sell','EURAUD_Open','AUDNZD_High','AUDNZD_Low']])),
                np.transpose(np.array(data[['NZDCAD_Open','NZDCAD_buy','NZDCAD_sell','EURNZD_Open','NZDCAD_High','NZDCAD_Low']])),
                np.transpose(np.array(data[['AUDCAD_Open','AUDCAD_buy','AUDCAD_sell','EURAUD_Open','AUDCAD_High','AUDCAD_Low']])),
            ], dtype=np.float64)
    return array

def pct_data_signals_quantile(data, low, high):

    for i in range(len(ALL_PAIRS_BUY)):

        results_sell = deque()
        results_buy = deque()

        pair1 = data[SPLIT_PAIRS[i][0]].to_numpy()
        pair2 = data[SPLIT_PAIRS[i][1]].to_numpy()

        pair1_1 = data[SPLIT_PAIRS[i][0]].shift().to_numpy()
        pair2_1 = data[SPLIT_PAIRS[i][1]].shift().to_numpy()

        std1_pos = np.quantile(pair1, high)
        std1_neg = np.quantile(pair1, low)
        std2_pos = np.quantile(pair2, high)
        std2_neg = np.quantile(pair2, low)

        data[ALL_PAIRS_SELL[i]] = pd.Series((pair1 < std1_neg) & (pair2 > std2_pos))
        data[ALL_PAIRS_BUY[i]] = pd.Series((pair1 > std1_pos) & (pair2 < std2_neg))

    return data


@njit
def finance_calculation(balance, saldo_inicial, saldo_final, preco_eur, eur):

    initial_balance = 1000

    lot = initial_balance

    calc = balance - initial_balance

    #if balance > initial_balance:
        #lot = (balance + (calc * 0.99)) // 1000 * 1000

    comission = (lot//1000) * 0.1

    if eur:
        tot = (lot * (saldo_inicial - saldo_final)) / saldo_final
        tot2 = round(tot - comission,2)
        return (tot2 + balance), tot2
    else:
        tot = (lot * (saldo_inicial - saldo_final)) / saldo_final
        tot2 = round(tot / saldo_final / preco_eur - comission,2)
        return (tot2 + balance), tot2


@njit
def otimizado_tpsl(series, m1, tpsl, balance=1000):

    check_eur_jpy = np.array(
            [
                [1,0],
                [1,0],
                [1,0],
                [1,1],
                [1,0],
                [1,0],
                [1,0],
                [0,0],
                [0,0],
                [0,1],
                [0,0],
                [0,0],
                [0,0],
                [0,0],
                [0,1],
                [0,0],
                [0,0],
                [0,0],
                [0,1],
                [0,1],
                [0,1],
                [0,1],
                [0,0],
                [0,0],
                [0,0],
                [0,0],
                [0,0],
                [0,0],
            ], dtype=np.bool_
        )
    operacoes = np.zeros((28,6),dtype=np.float64)
    buy_sell = np.ones((28,2),dtype=np.bool_)
    #history = TRADE_HISTORY

    buy_ind, sell_ind, list_backtest_indi = 0, 0, 1

    buy_orders = np.zeros((400_000), dtype=np.float64)
    sell_orders = np.zeros((400_000), dtype=np.float64)

    balance_backtest = balance
    list_backtest = np.zeros((800_000), dtype=np.float64)
    list_backtest[0] = balance_backtest

    # Talvez arrumar? Esta muito feio

    tk_normal = tpsl / 10000
    sl_normal = tpsl / 20000
    tk_jpy = tpsl / 100
    sl_jpy = tpsl / 200

    for i in range(series[0][0].size): # 63k

        buy_result = 0
        sell_result = 0

        for h in range(len(series)): # 28

            for super in np.arange(m1[h][0][i].size):

                if series[h][1][i] and buy_sell[h][0]:
                    operacoes[h][2] = series[h][0][i]
                    buy_sell[h][0] = False
                    if check_eur_jpy[h][1]:
                        operacoes[h][6] = series[h][0][i] + tk_jpy
                        operacoes[h][7] = series[h][0][i] - sl_jpy
                    else:
                        operacoes[h][6] = series[h][0][i] + tk_normal
                        operacoes[h][7] = series[h][0][i] - sl_normal

                check_buy_sl = (m1[h][0][i][super] <= operacoes[h][7]) or (m1[h][1][i][super] <= operacoes[h][7]) or (m1[h][2][i][super] <= operacoes[h][7]) or (m1[h][3][i][super] <= operacoes[h][7])
                check_buy_tp = (m1[h][0][i][super] >= operacoes[h][6]) or (m1[h][1][i][super] >= operacoes[h][6]) or (m1[h][2][i][super] >= operacoes[h][6]) or (m1[h][3][i][super] >= operacoes[h][6])

                if buy_sell[h][0] == False and check_buy_sl:
                    balance_backtest, buy_result = finance_calculation(balance=balance_backtest, saldo_inicial=operacoes[h][7],
                                                                       saldo_final=operacoes[h][2], eur = check_eur_jpy[h][0],
                                                                       preco_eur=series[h][3][i])
                    list_backtest[list_backtest_indi] = balance_backtest
                    buy_orders[buy_ind] = buy_result
                    buy_ind += 1
                    list_backtest_indi += 1
                    buy_sell[h][0] = True
                elif buy_sell[h][0] == False and check_buy_tp:
                    balance_backtest, buy_result = finance_calculation(balance=balance_backtest, saldo_inicial=operacoes[h][6],
                                                                       saldo_final=operacoes[h][2], eur = check_eur_jpy[h][0],
                                                                       preco_eur=series[h][3][i])
                    list_backtest[list_backtest_indi] = balance_backtest
                    buy_orders[buy_ind] = buy_result
                    buy_ind += 1
                    list_backtest_indi += 1
                    buy_sell[h][0] = True

                if series[h][2][i] and buy_sell[h][1]:
                    operacoes[h][3] = series[h][0][i]
                    buy_sell[h][1] = False
                    if check_eur_jpy[h][1]:
                        operacoes[h][4] = series[h][0][i] - tk_jpy
                        operacoes[h][5] = series[h][0][i] + sl_jpy
                    else:
                        operacoes[h][4] = series[h][0][i] - tk_normal
                        operacoes[h][5] = series[h][0][i] + sl_normal

                check_sell_sl = (m1[h][0][i][super] >= operacoes[h][5]) or (m1[h][1][i][super] >= operacoes[h][5]) or (m1[h][2][i][super] >= operacoes[h][5]) or (m1[h][3][i][super] >= operacoes[h][5])
                check_sell_buy = (m1[h][0][i][super] <= operacoes[h][4]) or (m1[h][1][i][super] <= operacoes[h][4]) or (m1[h][2][i][super] <= operacoes[h][4]) or (m1[h][3][i][super] <= operacoes[h][4])

                if buy_sell[h][1] == False and check_sell_sl:
                    balance_backtest, sell_result = finance_calculation(balance=balance_backtest,saldo_inicial=operacoes[h][3],
                                                                        saldo_final=operacoes[h][5], eur = check_eur_jpy[h][0],
                                                                        preco_eur=series[h][3][i])
                    list_backtest[list_backtest_indi] = balance_backtest
                    sell_orders[sell_ind] = sell_result
                    sell_ind += 1
                    list_backtest_indi += 1
                    buy_sell[h][1] = True

                elif buy_sell[h][1] == False and check_sell_buy:
                    balance_backtest, sell_result = finance_calculation(balance=balance_backtest, saldo_inicial=operacoes[h][3],
                                                                        saldo_final=operacoes[h][4], eur = check_eur_jpy[h][0],
                                                                        preco_eur=series[h][3][i])
                    list_backtest[list_backtest_indi] = balance_backtest
                    sell_orders[sell_ind] = sell_result
                    sell_ind += 1
                    list_backtest_indi += 1
                    buy_sell[h][1] = True

    return list_backtest, sell_orders, buy_orders

## Backtest

In [ ]:
datah1 = pct_data(datah1)

In [ ]:
datah1 = pct_data_signals_quantile(datah1, low=0.45, high=0.55)

In [ ]:
datah1 = pandas_to_array(datah1)

In [ ]:
datam1 = pandas_to_array_massive(datam1)

In [ ]:
datam1 = np.array([
              [
               np.array_split(datam1[0][0], 63000),
               np.array_split(datam1[0][1], 63000),
               np.array_split(datam1[0][2], 63000),
               np.array_split(datam1[0][3], 63000)
              ],
              [
               np.array_split(datam1[1][0], 63000),
               np.array_split(datam1[1][1], 63000),
               np.array_split(datam1[1][2], 63000),
               np.array_split(datam1[1][3], 63000)
              ],
              [
               np.array_split(datam1[2][0], 63000),
               np.array_split(datam1[2][1], 63000),
               np.array_split(datam1[2][2], 63000),
               np.array_split(datam1[2][3], 63000)
              ],
              [
               np.array_split(datam1[3][0], 63000),
               np.array_split(datam1[3][1], 63000),
               np.array_split(datam1[3][2], 63000),
               np.array_split(datam1[3][3], 63000)
              ],
              [
               np.array_split(datam1[4][0], 63000),
               np.array_split(datam1[4][1], 63000),
               np.array_split(datam1[4][2], 63000),
               np.array_split(datam1[4][3], 63000)
              ],
              [
               np.array_split(datam1[5][0], 63000),
               np.array_split(datam1[5][1], 63000),
               np.array_split(datam1[5][2], 63000),
               np.array_split(datam1[5][3], 63000)
              ],
              [
               np.array_split(datam1[6][0], 63000),
               np.array_split(datam1[6][1], 63000),
               np.array_split(datam1[6][2], 63000),
               np.array_split(datam1[6][3], 63000)
              ],
              [
               np.array_split(datam1[7][0], 63000),
               np.array_split(datam1[7][1], 63000),
               np.array_split(datam1[7][2], 63000),
               np.array_split(datam1[7][3], 63000)
              ],
              [
               np.array_split(datam1[8][0], 63000),
               np.array_split(datam1[8][1], 63000),
               np.array_split(datam1[8][2], 63000),
               np.array_split(datam1[8][3], 63000)
              ],
              [
               np.array_split(datam1[9][0], 63000),
               np.array_split(datam1[9][1], 63000),
               np.array_split(datam1[9][2], 63000),
               np.array_split(datam1[9][3], 63000)
              ],
              [
               np.array_split(datam1[10][0], 63000),
               np.array_split(datam1[10][1], 63000),
               np.array_split(datam1[10][2], 63000),
               np.array_split(datam1[10][3], 63000)
              ],
              [
               np.array_split(datam1[11][0], 63000),
               np.array_split(datam1[11][1], 63000),
               np.array_split(datam1[11][2], 63000),
               np.array_split(datam1[11][3], 63000)
              ],
              [
               np.array_split(datam1[12][0], 63000),
               np.array_split(datam1[12][1], 63000),
               np.array_split(datam1[12][2], 63000),
               np.array_split(datam1[12][3], 63000)
              ],
              [
               np.array_split(datam1[13][0], 63000),
               np.array_split(datam1[13][1], 63000),
               np.array_split(datam1[13][2], 63000),
               np.array_split(datam1[13][3], 63000)
              ],
              [
               np.array_split(datam1[14][0], 63000),
               np.array_split(datam1[14][1], 63000),
               np.array_split(datam1[14][2], 63000),
               np.array_split(datam1[14][3], 63000)
              ],
              [
               np.array_split(datam1[15][0], 63000),
               np.array_split(datam1[15][1], 63000),
               np.array_split(datam1[15][2], 63000),
               np.array_split(datam1[15][3], 63000)
              ],
              [
               np.array_split(datam1[16][0], 63000),
               np.array_split(datam1[16][1], 63000),
               np.array_split(datam1[16][2], 63000),
               np.array_split(datam1[16][3], 63000)
              ],
              [
               np.array_split(datam1[17][0], 63000),
               np.array_split(datam1[17][1], 63000),
               np.array_split(datam1[17][2], 63000),
               np.array_split(datam1[17][3], 63000)
              ],
              [
               np.array_split(datam1[18][0], 63000),
               np.array_split(datam1[18][1], 63000),
               np.array_split(datam1[18][2], 63000),
               np.array_split(datam1[18][3], 63000)
              ],
              [
               np.array_split(datam1[19][0], 63000),
               np.array_split(datam1[19][1], 63000),
               np.array_split(datam1[19][2], 63000),
               np.array_split(datam1[19][3], 63000)
              ],
              [
               np.array_split(datam1[20][0], 63000),
               np.array_split(datam1[20][1], 63000),
               np.array_split(datam1[20][2], 63000),
               np.array_split(datam1[20][3], 63000)
              ],
              [
               np.array_split(datam1[21][0], 63000),
               np.array_split(datam1[21][1], 63000),
               np.array_split(datam1[21][2], 63000),
               np.array_split(datam1[21][3], 63000)
              ],
              [
               np.array_split(datam1[22][0], 63000),
               np.array_split(datam1[22][1], 63000),
               np.array_split(datam1[22][2], 63000),
               np.array_split(datam1[22][3], 63000)
              ],
              [
               np.array_split(datam1[23][0], 63000),
               np.array_split(datam1[23][1], 63000),
               np.array_split(datam1[23][2], 63000),
               np.array_split(datam1[23][3], 63000)
              ],
              [
               np.array_split(datam1[24][0], 63000),
               np.array_split(datam1[24][1], 63000),
               np.array_split(datam1[24][2], 63000),
               np.array_split(datam1[24][3], 63000)
              ],
              [
               np.array_split(datam1[25][0], 63000),
               np.array_split(datam1[25][1], 63000),
               np.array_split(datam1[25][2], 63000),
               np.array_split(datam1[25][3], 63000)
              ],
              [
               np.array_split(datam1[26][0], 63000),
               np.array_split(datam1[26][1], 63000),
               np.array_split(datam1[26][2], 63000),
               np.array_split(datam1[26][3], 63000)
              ],
              [
               np.array_split(datam1[27][0], 63000),
               np.array_split(datam1[27][1], 63000),
               np.array_split(datam1[27][2], 63000),
               np.array_split(datam1[27][3], 63000)
              ]
])


In [ ]:
%%time
result, _, _ = otimizado_tpsl(datah1, datam1, 50)

In [ ]:
pd.Series(result).plot()